In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

import core.config as cconfig
import core.plotting as coplotti
import dataflow.model as dtfmod
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hparquet as hparque
import helpers.hprint as hprint

/app/amp/core/explore.py:39: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tauton


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-9fcd3976-7603-42b1-b634-ea3abce1b415.json'
>>ENV<<: is_inside_container=True: code_version=1.0.3, container_version=amp-1.0.3, is_inside_docker=True, is_inside_ci=False, CI_defined=True, CI=''
>>ENV<<: AM_AWS_PROFILE=True AM_ECR_BASE_PATH=True AM_S3_BUCKET=True AM_TELEGRAM_TOKEN=True AWS_ACCESS_KEY_ID=False AWS_DEFAULT_REGION=False AWS_SECRET_ACCESS_KEY=False GH_ACTION_ACCESS_TOKEN=True
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.0.3' != container_version='amp-1.0.3'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
    branch_name='AmpTask2158_Add_stats_comparison_nb_03'
    hash='82edfda37'
    # Last com

# Load tiled backtest

In [7]:
dict_ = {
    "file_name": "",
    "asset_id_col": "",
    "prediction_col": "",
    "returns_col": "",
    "volatility_col": "",
    "target_gmv": 1e6,
    "dollar_neutrality": "no_constraint",
    "freq": "5T",
    # "start_timestamp": "",
    # "end_timestamp": "",
}

In [8]:
config = cconfig.get_config_from_nested_dict(dict_)

In [10]:
columns = [
    config["asset_id_col"],
    config["prediction_col"],
    config["returns_col"],
    config["volatility_col"],
]

In [ ]:
parquet_df = hparque.from_parquet(config["file_name"], columns=columns)

In [ ]:
df = dtfmod.process_parquet_read_df(parquet_df, config["asset_id_col"])

In [ ]:
_LOG.info("num_assets=%d", len(df.columns.levels[1]))

# Computer per-bar portfolio stats

In [ ]:
forecast_evaluator = dtfmod.ForecastEvaluator(
    returns_col=config["returns_col"],
    volatility_col=config["volatility_col"],
    prediction_col=config["prediction_col"],
)

In [ ]:
portfolio_df, bar_stats_df = forecast_evaluator.annotate_forecasts(
    df,
    target_gmv=config["target_gmv"],
    dollar_neutrality=config["dollar_neutrality"],
)

# Computer aggregate portfolio stats

In [ ]:
stats_computer = dtfmod.StatsComputer()

In [ ]:
stats, stats_df = stats_computer.compute_portfolio_stats(
    bar_stats_df, config["freq"]
)
display(stats)

# Plot portfolio metrics

In [ ]:
coplotti.plot_portfolio_stats(bar_stats_df)